## Importing libraries

In [1]:
import numpy as np 
import numpy.linalg as LA
import scipy.stats as stat
from time import time
import os

from scipy.special import betainc
import tensorflow as tf
physical_devices=tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(device=physical_devices[0],enable=True)
import tensorflow_datasets as tfds
from tensorflow_probability import distributions as tfd

In [80]:
import foolbox as fb

In [2]:
tf.config.list_physical_devices()

[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [3]:
import langevin_smc as smc
import sampling_tools as s_t
from utils import normal_kernel
from importlib import reload
reload(smc)
reload(s_t)

<module 'sampling_tools' from '/home/karim-tito/sampling-reliability-measure/karimtito_reliability_meassure/sampling_tools.py'>

# Neural Network Model tests for vanilla Langevin SMC-based reliability measure

## 1. Toy model on MNIST 

Let $f$ be neural network model and $x_0$ be an input of class index $c$, correctly classified by $f$.
We want to estimate the local failure probability $p$, defined by, 
$$p=\mathbb{P}[h(x_0+X)\geq 0]$$
where $h:x\mapsto \max_{j\neq c}f_j(x)-f_c(x)$ and $X\sim \mathcal{U}([-\varepsilon,\varepsilon]^d)$ under measure $\mathbb{P}$. 
We are interested in the case where $p$ is especially small (say $<10^{-10}$).
We define the potential function $V$, 
$$V:x\mapsto (f_c(x_0+x)-\max_{j\neq c}f_j(x_0+x))_+$$
So that $p=\mathbb{P}[V(X)=0]$. For $\beta \in \R_+ \cup \{+\infty\}$, we denote $\mu_{\beta}$ the probability measure on $[-\varepsilon,\varepsilon]^d$ proportional to $e^{-\beta V(x)}dx$ and we denote $Z_{\beta}$ it's normalization constant. Then, one can notice that, 
$$Z_{\infty} = \int_{[-\varepsilon,\varepsilon]^d} \mathbb{1}_{V(x)=0}dx=p


## 2. Loading and pre-processing MNIST data

In [4]:
nb_classes=10
ds_train, ds_val, ds_test = tfds.load('mnist',split=('train[:95%]','train[95%:]','test'),as_supervised=True)

def label_to_one_hot(data, label):
    return data, tf.one_hot(label,nb_classes,on_value=1.0,off_value=0.0)
batch_size=64
ds_train=ds_train.map(lambda data,label: (data/255,label)).map(label_to_one_hot).batch(batch_size)
ds_val=ds_val.map(lambda data,label: (data/255,label)).map(label_to_one_hot).batch(batch_size)
ds_test=ds_test.map(lambda data,label: (data/255,label)).map(label_to_one_hot).batch(batch_size)



## 3. Creating DNN toy models

In [6]:
nets_dir= "/home/karim-tito/sampling-reliability-measure/karimtito_reliability_meassure/models/mnist"
net_paths=[]

In [11]:
net_path=os.path.join(nets_dir,'dnn')
net_paths.append(net_path)
dnn = tf.keras.Sequential()
dnn.add(tf.keras.layers.Flatten())
dnn.add(tf.keras.layers.Dense(100,activation='relu'))
dnn.add(tf.keras.layers.Dense(100,activation ='relu'))
dnn.add(tf.keras.layers.Dense(100,activation='relu'))
dnn.add(tf.keras.layers.Dense(10))

model_dnn_2 = tf.keras.Sequential()
model_dnn_2.add(tf.keras.layers.Flatten())
model_dnn_2.add( tf.keras.layers.Dense(200,'relu'))
model_dnn_2.add(tf.keras.layers.Dense(10,))

In [8]:
model_dnn_4 = tf.keras.Sequential()
model_dnn_4.add(tf.keras.layers.Flatten())
model_dnn_4.add( tf.keras.layers.Dense(200,'relu'))
model_dnn_4.add(
                            tf.keras.layers.Dense(100,'relu'))
model_dnn_4.add(    tf.keras.layers.Dense(100,'relu'))
model_dnn_4.add(        tf.keras.layers.Dense(10))

x = tf.keras.Input(shape = (28,28,1))
output = tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu')(x)
output = tf.keras.layers.Conv2D(filters=32,kernel_size=3,padding='same',activation='relu',strides=2)(output)
output=tf.keras.layers.Conv2D(filters=64,kernel_size=3, padding='same',
activation='relu')(output)
output=tf.keras.layers.Conv2D(filters=64,kernel_size=3, padding='same',strides=2,
activation='relu')(output)
output = tf.keras.layers.Flatten()(output)
output = tf.keras.layers.Dense(units=100,activation='relu')(output)
output = tf.keras.layers.Dense(units=10)(output)
model_cnn= tf.keras.Model(inputs=x, outputs=output)
model_cnn.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics='accuracy')

## 4. Training the DNNs 

In [9]:
model_dnn_2.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-1),
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics='accuracy')
model_dnn_2.fit(ds_train,epochs=10,verbose=1)

Epoch 1/10
891/891 [==============================] - 2s 2ms/step - loss: 0.3747 - accuracy: 0.8966
Epoch 2/10
891/891 [==============================] - 1s 1ms/step - loss: 0.1958 - accuracy: 0.9444
Epoch 3/10
891/891 [==============================] - 1s 1ms/step - loss: 0.1467 - accuracy: 0.9585
Epoch 4/10
891/891 [==============================] - 1s 1ms/step - loss: 0.1181 - accuracy: 0.9670
Epoch 5/10
891/891 [==============================] - 1s 2ms/step - loss: 0.0989 - accuracy: 0.9732
Epoch 6/10
891/891 [==============================] - 1s 2ms/step - loss: 0.0849 - accuracy: 0.9770
Epoch 7/10
891/891 [==============================] - 1s 1ms/step - loss: 0.0741 - accuracy: 0.9800
Epoch 8/10
891/891 [==============================] - 1s 1ms/step - loss: 0.0655 - accuracy: 0.9822
Epoch 9/10
891/891 [==============================] - 1s 1ms/step - loss: 0.0584 - accuracy: 0.9841
Epoch 10/10
891/891 [==============================] - 1s 1ms/step - loss: 0.0524 - accuracy: 0.9860

In [10]:
nets_dir =  '/home/karim-tito/sampling-reliability-measure/karimtito_reliability_meassure/models/mnist'

In [11]:
model_dnn_2.evaluate(ds_test)

tf.keras.models.save_model(model=model_dnn_2,filepath=os.path.join(nets_dir,"dnn_2.h5"),save_format='h5')

157/157 [==============================] - 1s 2ms/step - loss: 0.0747 - accuracy: 0.9760


In [38]:
model_cnn.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-1),
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics='accuracy')
model_cnn.fit(ds_train,epochs=4,verbose=1)
model_cnn.evaluate(ds_test)
tf.keras.models.save_model(model=model_cnn,filepath=os.path.join(nets_dir,"cnn.h5"),save_format='h5')

Epoch 1/4
891/891 [==============================] - 3s 3ms/step - loss: 0.0222 - accuracy: 0.9935
Epoch 2/4
891/891 [==============================] - 3s 3ms/step - loss: 0.0174 - accuracy: 0.9945
Epoch 3/4
891/891 [==============================] - 3s 3ms/step - loss: 0.0157 - accuracy: 0.9941
Epoch 4/4
157/157 [==============================] - 0s 2ms/step - loss: 0.0552 - accuracy: 0.9882


In [39]:

model_dnn_4.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=2e-3),
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics='accuracy')
model_dnn_4.fit(ds_train,epochs=8,verbose=1)
model_dnn_4.evaluate(ds_test)
tf.keras.models.save_model(model=model_dnn_4,filepath=os.path.join(nets_dir,"dnn_4.h5"),save_format='h5')

Epoch 1/8
891/891 [==============================] - 2s 2ms/step - loss: 0.0170 - accuracy: 0.9956
Epoch 2/8
891/891 [==============================] - 2s 2ms/step - loss: 0.0165 - accuracy: 0.9955
Epoch 3/8
891/891 [==============================] - 2s 2ms/step - loss: 0.0129 - accuracy: 0.9965
Epoch 4/8
891/891 [==============================] - 2s 2ms/step - loss: 0.0141 - accuracy: 0.9961
Epoch 5/8
891/891 [==============================] - 2s 2ms/step - loss: 0.0130 - accuracy: 0.9959
Epoch 6/8
891/891 [==============================] - 2s 2ms/step - loss: 0.0133 - accuracy: 0.9964
Epoch 7/8
891/891 [==============================] - 2s 2ms/step - loss: 0.0138 - accuracy: 0.9959
Epoch 8/8
157/157 [==============================] - 1s 3ms/step - loss: 0.1374 - accuracy: 0.9826


In [40]:
model_dnn_4.evaluate(ds_test)


157/157 [==============================] - 1s 3ms/step - loss: 0.1374 - accuracy: 0.9826


[0.13741649687290192, 0.9825999736785889]

In [41]:
dnn.compile(optimizer=tf.keras.optimizers.SGD(learning_rate=1e-2),
loss=tf.keras.losses.CategoricalCrossentropy(from_logits=True),metrics='accuracy')
dnn.fit(ds_train,epochs=15,verbose=1)
dnn.evaluate(ds_test)


Epoch 1/15
891/891 [==============================] - 3s 3ms/step - loss: 0.0520 - accuracy: 0.9853
Epoch 2/15
891/891 [==============================] - 2s 2ms/step - loss: 0.0500 - accuracy: 0.9862
Epoch 3/15
891/891 [==============================] - 2s 2ms/step - loss: 0.0480 - accuracy: 0.9864
Epoch 4/15
891/891 [==============================] - 2s 2ms/step - loss: 0.0461 - accuracy: 0.9870
Epoch 5/15
891/891 [==============================] - 2s 2ms/step - loss: 0.0443 - accuracy: 0.9875
Epoch 6/15
891/891 [==============================] - 2s 2ms/step - loss: 0.0426 - accuracy: 0.9881
Epoch 7/15
891/891 [==============================] - 2s 2ms/step - loss: 0.0409 - accuracy: 0.9890
Epoch 8/15
891/891 [==============================] - 2s 2ms/step - loss: 0.0393 - accuracy: 0.9895
Epoch 9/15
891/891 [==============================] - 2s 3ms/step - loss: 0.0377 - accuracy: 0.9899
Epoch 10/15
891/891 [==============================] - 2s 2ms/step - loss: 0.0362 - accuracy: 0.9907

[0.08408880233764648, 0.9761000275611877]

In [42]:
tf.keras.models.save_model(model=dnn,filepath=os.path.join(nets_dir,"dnn.h5"),save_format='h5')

## (3+4)#. Loading the DNNs

In [5]:
nets_dir =  '/home/karim-tito/sampling-reliability-measure/karimtito_reliability_meassure/models/mnist'
model_dnn_2 = tf.keras.models.load_model(filepath=os.path.join(nets_dir,"dnn_2.h5"))
model_dnn_4 = tf.keras.models.load_model(filepath=os.path.join(nets_dir,"dnn_4.h5"))
dnn= tf.keras.models.load_model(filepath=os.path.join(nets_dir,"dnn.h5"))
model_cnn = tf.keras.models.load_model(filepath=os.path.join(nets_dir,"cnn.h5"))

In [6]:
models=[model_dnn_2, dnn,model_dnn_4, model_cnn]
model_names=['dnn_2','dnn_3','dnn_4','cnn']

## 5. Instancing potential and potential gradient functions

In [85]:
l=40
test_model= model_cnn
for X,y in ds_test.take(1):
    logits = test_model(X)
    y_true= tf.argmax(y,-1)

y_pred= tf.argmax(logits,-1)
correct_idx=y_pred==y_true

x_0,y_0 = X[correct_idx][l], tf.cast(y_true[correct_idx][l],tf.int32)
X_correct, label_correct= X[correct_idx], tf.cast(y_true[correct_idx],tf.int32)

In [59]:
normal_dist=tfd.Normal(loc=0,scale=1)

def V(x_,x_0,model,target_class,epsilon=0.05,gaussian_latent_var=True,clipping=True, clip_min=0, clip_max=1,reshape=True,input_shape=(28,28,1)):
    if gaussian_latent_var:
        u=epsilon*(2*normal_dist.cdf(x_)-1)
    else:
        u=x_
    if reshape:
        u=tf.reshape(u,(u.shape[0],)+input_shape)
    x_p = x_0+u if not clipping else tf.clip_by_value(x_0+u,clip_value_min=clip_min,clip_value_max=clip_max)
    v = smc.compute_V_tf(model=model,input=x_p,target_class=target_class)
    return v

def gradV(x_,x_0,model,target_class,epsilon=0.05,gaussian_latent_var=True,clipping=True, clip_min=0, clip_max=1,reshape=True,input_shape=(28,28,1)):
    if gaussian_latent_var:
        u=epsilon*(2*normal_dist.cdf(x_)-1)
    else:
        u=x_
    if reshape:
        u=tf.reshape(u,(u.shape[0],)+input_shape)
    x_p = x_0+u if not clipping else tf.clip_by_value(x_0+u,clip_value_min=clip_min,clip_value_max=clip_max)
    _,grad_u = smc.compute_V_grad_tf(model=model,input=x_p,target_class=target_class)
    grad_u=tf.reshape(grad_u,x_.shape)
    if gaussian_latent_var:
        #TODO
        grad_x=normal_dist.prob(x_)*grad_u/(2*epsilon)
    else:
        grad_x=grad_u
    return grad_x

In [76]:
epsilon=0.0001
x_ =tf.random.normal((100,784))
v=V(x_=x_,x_0=x_0,model=test_model,target_class=y_0,epsilon=epsilon)
lr=5
grad_v=gradV(x_=x_,x_0=x_0,model=test_model,target_class=y_0,epsilon=epsilon)
x__=tf.clip_by_value(x_-lr*grad_v,clip_value_min=0,clip_value_max=1)
v_=V(x_=x__,x_0=x_0,model=test_model,target_class=y_0,epsilon=epsilon)
v_-v

<tf.Tensor: shape=(100,), dtype=float32, numpy=
array([-0.01006413, -0.01109028, -0.01189899, -0.01058006, -0.01024437,
       -0.00877285, -0.01077366, -0.00924873, -0.01157188, -0.01019382,
       -0.0111742 , -0.01180172, -0.00935078, -0.01048946, -0.01026821,
       -0.01064396, -0.01058769, -0.01050854, -0.01135159, -0.01064968,
       -0.01039219, -0.01129913, -0.01077652, -0.01064968, -0.01046658,
       -0.01055241, -0.01133442, -0.0100565 , -0.01137257, -0.00903034,
       -0.01144981, -0.01171589, -0.00998592, -0.00992298, -0.0102663 ,
       -0.01238537, -0.01023102, -0.01086044, -0.01083565, -0.01243877,
       -0.0119915 , -0.00958824, -0.00874138, -0.00977802, -0.01079655,
       -0.01029301, -0.00942802, -0.01088905, -0.01074028, -0.01154327,
       -0.01153278, -0.01115513, -0.01093102, -0.00808716, -0.01019287,
       -0.00948524, -0.00980854, -0.01103592, -0.00991344, -0.01237488,
       -0.00997829, -0.0093708 , -0.01264286, -0.00900078, -0.01134205,
       -0.010476

## 6. Testing Langevin SMC

In [75]:
reload(smc)

<module 'langevin_smc' from '/home/karim-tito/sampling-reliability-measure/karimtito_reliability_meassure/langevin_smc.py'>

In [78]:
epsilons=[0.001,0.01,0.05,0.08,0.1,0.2,0.3]
p_ests=[]
times=[]
for epsilon in epsilons[1:]:
    normal_gen=lambda N: tf.random.normal((N,784))
    V_ = lambda X: V(X,x_0=x_0,model=test_model,epsilon=epsilon, target_class=y_0)
    gradV_ = lambda X: gradV(X,x_0=x_0,model=test_model, target_class=y_0,epsilon=epsilon)
    t=time()
    p_est=smc.LangevinSMCBaseTF(gen=normal_gen, l_kernel=smc.langevin_kernel,V=V_, gradV=gradV_,rho=500,beta_0=0, min_rate=0.5,
    alpha =1,N=50,T = 1,n_max=10000, verbose=2,adapt_func=None,
    step_decay=0.0)
    p_ests.append(p_est)
    t=time()-t
    times.append(t)


Initial time step: dt=0.02413099631667137
Iter =  1  v_mean =  7.225357  Calls =  150 v_std =  0.09434213
New time step: dt=0.02413099631667137
Iter =  2  v_mean =  6.6601434  Calls =  250 v_std =  0.09266721
New time step: dt=0.02413099631667137
Iter =  3  v_mean =  6.2632966  Calls =  350 v_std =  0.08701106
New time step: dt=0.02413099631667137
Iter =  4  v_mean =  5.9873314  Calls =  450 v_std =  0.083386675
New time step: dt=0.02413099631667137
Iter =  5  v_mean =  5.7857146  Calls =  550 v_std =  0.07702056
New time step: dt=0.02413099631667137
Iter =  6  v_mean =  5.6323705  Calls =  650 v_std =  0.06944057
New time step: dt=0.02413099631667137
Iter =  7  v_mean =  5.515689  Calls =  750 v_std =  0.061810046
New time step: dt=0.02413099631667137
Iter =  8  v_mean =  5.42567  Calls =  850 v_std =  0.054019623
New time step: dt=0.02413099631667137
Iter =  9  v_mean =  5.3550663  Calls =  950 v_std =  0.047631036
New time step: dt=0.02413099631667137
Iter =  10  v_mean =  5.297968 

RuntimeError: The estimator failed. Increase n_max?

## 7. Adversarial attacks comparison


In [81]:
fmodel = fb.TensorFlowModel(model=test_model, bounds=(0,1))


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


Instructions for updating:
Use `tf.config.list_physical_devices('GPU')` instead.


In [84]:
attack =fb.attacks.LinfPGD()

In [90]:
epsilons = [0.001, 0.005, 0.01, 0.03, 0.1, 0.3, 0.5, 1.0]
_, advs, success = attack(fmodel, X_correct, label_correct, epsilons=epsilons)

In [91]:
success

<tf.Tensor: shape=(8, 62), dtype=bool, numpy=
array([[False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, False, False, False, False, False,
        False, False, False, False, F

## 8. Test last particle

In [29]:
reload(smc)

<module 'langevin_smc' from '/home/karim-tito/sampling-reliability-measure/karimtito_reliability_meassure/langevin_smc.py'>

In [30]:
def h_(x):
    u = epsilon*(2*normal_dist.cdf(x)-1)
    u=tf.reshape(u,(x.shape[0],28,28,1)) if len(x.shape)>1 else tf.reshape(u,(1,28,28,1))
    u=tf.clip_by_value(u, clip_value_max=1.0,clip_value_min=0.)
    h = smc.compute_h_tf(model=test_model, input=x_0+u,target_class=y_0)
    return h.numpy()

In [31]:
gaussian_gen= lambda N: np.random.normal(size=(N,784))

In [32]:
p_est,s_out= s_t.ImportanceSplittingLp(gen=gaussian_gen,kernel=normal_kernel,h=h_,tau=0,N=2,s=0.1,decay=0.9,T = 20, accept_ratio = 0.9, 
alpha_est = 0.95, alpha_test=0.99,verbose=1, gain_thresh=0.01, check_every=3, p_c = 10**(-20),n_max = int(10**6), 
  reject_forget_rate =0, gain_forget_rate=0, reject_thresh=0.005)

Starting Last Particle algorithm with 116, to certify p<p_c=1e-20, with confidence level alpha =0.010000000000000009.
Iter =  1  tau_j =  -15.147199  Calls =  2
Accept ratio:0.85
Reject rate:0.05000000000000001
Iter =  4  tau_j =  -14.61069  Calls =  62
Accept ratio:0.95
Reject rate:0.07500000000000002
Iter =  7  tau_j =  -14.415869  Calls =  122
Accept ratio:0.9
Reject rate:0.10555555555555557
Strength of kernel diminished!
s=0.09000000000000001
Iter =  10  tau_j =  -14.22985  Calls =  182
Accept ratio:0.4
Reject rate:0.13333333333333336
Strength of kernel diminished!
s=0.08100000000000002
Iter =  13  tau_j =  -13.941614  Calls =  242
Accept ratio:1.0
Reject rate:0.11000000000000006
Strength of kernel diminished!
s=0.05904900000000002
Iter =  16  tau_j =  -13.762367  Calls =  302
Accept ratio:1.0
Reject rate:0.09444444444444448
Iter =  19  tau_j =  -13.54697  Calls =  362
Accept ratio:0.75
Reject rate:0.10000000000000003
Strength of kernel diminished!
s=0.043046721000000024
Iter =  22